In [7]:
from emlib import emlib

In [2]:
def NPZB_int(t,initial,dtinput,constants):

    P = initial[0]             # phytoplankton
    Z = initial[1]             # zooplankton
    B = initial[2]             # benthos
    DIN = initial[3]           # dissolved inorganic
    DON = initial[4]           # dissolved organic
    ddin = dtinput.Val("ddin")
    ddon = dtinput.Val("ddon")
    
    i = constants.Val("i")      #irradiance
    Pg = constants.Val("Pg")    # max P growth
    Bg = constants.Val("Bg")    # max B growth
    e = constants.Val("e")      # Z ingestion
    Zm = constants.Val("Zm")    # Z mortality
    Pm = constants.Val("Pm")    # P mortality
    Bm = constants.Val("Bm")    # B mortality
    Pe = constants.Val("Pe")    # P excretion
    Be = constants.Val("Be")    # B excretion
    Pr = constants.Val("Pr")    # P recycle
    h = constants.Val("h")      # Z grazing
    s = constants.Val("s")      # sedimentation
    
    P_dot = (i * Pg * P) + (Pr * DON * P) - (h * P * Z) - (Pm * P * P) - (Pe * P)
    Z_dot = ((2 * e - 1)*h * P * Z) - (Zm * Z * Z)
    B_dot = (Bg * B)/ (1 + Bg * B) * (DON + DIN) - (BM * B * B) - (Be * B)
    DIN_dot = ddin + (0.8* (1 - e) * h * P * Z) + (Be * B) - (i * Pg * P) - (Bg * B)/ (1 + Bg * B) * DIN
    DON_dot = ddon + (Bm * B * B) + (Pm * P * P) + (Zm * Z * Z) + (Pe * P) + (0.2 * (1 - e) * H * P * Z) \
             - (Pr * DON * P) - ((Bg * B)/ (1 + Bg * B) * DON) - (s * DON)

    return [P_dot, Z_dot, B_dot, DIN_dot, DON_dot]

In [7]:
NPZB = emlib.Model(NPZB_int) #save the model

INFO -1105- New Model(1): NPZB_int
INFO -1117- No algorithm supplied assuming vode/bfd O12 Nsteps3000 dt1
DEBUG -1131- Integrator:vode/bdf order:12 nsteps:3000 dt:1


In [12]:
GE_Zoop= emlib.TimeSeries(dirname="Data",filename="GE_Zoop.csv")

INFO -880- New TimeSeries instance: 4
DEBUG -952- New INPUT table DataGE_Zoop.csv['Date', 'Sta', 'Rep', 'Density']
DEBUG -963- Saved 60 rows and 3 columns
DEBUG -965- Converted dates to contiguous np.array
DEBUG -967- Converted input data to contiguous np.array
